In [1]:
import json
from pathlib import Path
import pandas as pd 

In [2]:
from bellem.dvc.experiment import load_experiments

filepaths = list(Path("../../tmp/musique-temperature-sweep/").glob("*.json"))
experiments = [exp for fp in filepaths for exp in load_experiments(fp)]
print(f"{len(experiments)} experiments")
experiments[0]

114 experiments


{'commit': 'workspace',
 'id': 'workspace',
 'name': None,
 'params': {'train': {'dataset': {'path': 'bdsaglam/musique-mini',
    'name': 'answerable',
    'split': 'train'},
   'optimizer': 'noop'},
  'evaluation': {'dataset': {'path': 'bdsaglam/musique',
    'name': 'answerable',
    'split': 'validation'}},
  'qa': {'model': 'llama-3-70b-tgi',
   'temperature': 0.1,
   'technique': 'standard'},
  'run': 1},
 'metrics': {}}

In [3]:
df = pd.json_normalize(experiments).drop(columns=["commit"])
mask = (
    (df["params.evaluation.dataset.path"] == "bdsaglam/musique")
    & (df["params.qa.temperature"] > 0.0)
    & (df["params.qa.temperature"] < 1.0)
)

df = df.loc[mask].copy()
df.drop(columns=[col for col in df.columns if "fuzzy" in col], inplace=True)

param_cols = [col for col in df.columns if col.startswith("params.")]
metric_cols = [col for col in df.columns if col.startswith("metrics.")]
df.dropna(subset=param_cols + metric_cols, inplace=True, how="any")
df.drop_duplicates(subset=param_cols, inplace=True)

print(f"{len(df)} experiments after preprocessing")

66 experiments after preprocessing


In [4]:
def sorted_tuple(x):
    return tuple(sorted(x))

run_counts = (
    df.groupby(param_cols[:-1])[
        "params.run"
    ]
    .aggregate(sorted_tuple)
    .reset_index()
)
run_counts.loc[run_counts["params.run"].map(len) != 6]

,params.train.dataset.path,params.train.dataset.name,params.train.dataset.split,params.train.optimizer,params.evaluation.dataset.path,params.evaluation.dataset.name,params.evaluation.dataset.split,params.qa.model,params.qa.temperature,params.qa.technique,params.run


In [5]:
for col in run_counts.columns:
    print(f"- {col}")
    for value in run_counts[col].unique():
        print(f"\t{value}")

- params.train.dataset.path
	bdsaglam/musique-mini
- params.train.dataset.name
	answerable
- params.train.dataset.split
	train
- params.train.optimizer
	bfsrs-medium
- params.evaluation.dataset.path
	bdsaglam/musique
- params.evaluation.dataset.name
	answerable
- params.evaluation.dataset.split
	validation
- params.qa.model
	llama-3-70b-tgi
- params.qa.temperature
	0.1
	0.5
	0.7
- params.qa.technique
	ccot
	cot
	cte
	standard
- params.run
	(1, 2, 3, 4, 5, 6)


In [6]:
df.to_json('results.jsonl', orient='records', lines=True)